In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
import pandas as pd

import keras
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.callbacks import EarlyStopping


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import ADASYN
import keras_costum_metric.Balanced_accuracy as ba
import keras_costum_metric.F1metric_keras as f1
import keras_custom_callback.Early_Stop_AccLoss as Stop_AL
import keras_custom_callback.Early_Stop_AccLoss as Checkpoint_A

import Set_model 
import plot
import sampling

In [12]:
# Load the dataset
credit_df=pd.read_csv("..\\data\\german_credit.csv")
pd.set_option('display.max_columns', None)
credit_df.head(10)

,Creditability,Account Balance,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,Duration in Current address,Most valuable available asset,Age (years),Concurrent Credits,Type of apartment,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
0,1,1,18,4,2,1049,1,2,4,2,1,4,2,21,3,1,1,3,1,1,1
1,1,1,9,4,0,2799,1,3,2,3,1,2,1,36,3,1,2,3,2,1,1
2,1,2,12,2,9,841,2,4,2,2,1,4,1,23,3,1,1,2,1,1,1
3,1,1,12,4,0,2122,1,3,3,3,1,2,1,39,3,1,2,2,2,1,2
4,1,1,12,4,0,2171,1,3,4,3,1,4,2,38,1,2,2,2,1,1,2
5,1,1,10,4,0,2241,1,2,1,3,1,3,1,48,3,1,2,2,2,1,2
6,1,1,8,4,0,3398,1,4,1,3,1,4,1,39,3,2,2,2,1,1,2
7,1,1,6,4,0,1361,1,2,2,3,1,4,1,40,3,2,1,2,2,1,2
8,1,4,18,4,3,1098,1,1,4,2,1,4,3,65,3,2,2,1,1,1,1
9,1,2,24,2,3,3758,3,1,1,2,1,4,4,23,3,1,1,1,1,1,1


In [13]:
X = credit_df.drop(columns=['Creditability'])
Y = credit_df['Creditability']

In [14]:
np.shape(X)

(1000, 20)

In [15]:
np.shape(Y)


(1000,)

In [16]:
scaler = MinMaxScaler()
scaler.fit(X)
t_X = scaler.transform(X)

In [17]:
print(t_X)


[[0.         0.20588235 1.         ... 0.         0.         0.        ]
 [0.         0.07352941 1.         ... 1.         0.         0.        ]
 [0.33333333 0.11764706 0.5        ... 0.         0.         0.        ]
 ...
 [1.         0.25       1.         ... 0.         1.         0.        ]
 [0.33333333 0.11764706 0.5        ... 0.         1.         0.        ]
 [0.         0.38235294 0.5        ... 0.         0.         0.        ]]


In [18]:
metrics=['accuracy', keras.metrics.Precision(), ba.BalancedAccuracy() , f1.F1Score(), keras.metrics.AUC(name='auc')]


In [19]:
loss = keras.losses.BinaryCrossentropy()

In [20]:
adasyn = ADASYN(random_state=42)

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Initialize metrics lists
accuracies = []
precisions = []
# recalls = []
f1_scores = []
roc_aucs = []
# true_negatives = []
balance_accuracies = []

# Perform KFold cross-validation
for train_index, test_index in kf.split(t_X, Y):
    X_train , X_test = t_X.iloc[train_index,:], t_X.iloc[test_index,:]
    y_train, y_test = Y[train_index], Y[test_index]

    X_adasyn, y_adasyn = adasyn.fit_resample(X_train_split, y_train_split)

    X_train_split, X_val, y_train_split, y_val = train_test_split(X_adasyn, y_adasyn, test_size=0.3, stratify=y_adasyn, random_state=200)

    base_model_num = 3

    models = []

    X_samples, y_samples = sampling.create_bootstrap_samples(X_train_split, y_train_split, base_model_num)

    for i in range(1,base_model_num + 1):
        adam = keras.optimizers.Adam(learning_rate=0.00005)

        model = Set_model.model1(optimizer=adam, metric=metrics, loss=loss)

        early_stopping = Stop_AL.EarlyStopAccLoss(
        min_delta=0.001,
        min_delta_loss = 0.1,
        patience_auc=100,
        verbose=1,
        )

        checkpoint = Checkpoint_A.CheckpointBestAUC(min_delta_loss=0.1, filepath = f'model\\best_model_03_{i}.keras')

        history = model.fit(X_samples[i], y_samples[i], epochs=500, callbacks = [early_stopping, checkpoint])

        models.append(model)

    

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'